### 1. Download Shopify dataset and create table script:

In [ ]:
!gsutil cp gs://cs327e-open-access/spanner_shopify.zip /home/jupyter/kimble-munroe

In [ ]:
!unzip /home/jupyter/kimble-munroe/spanner_shopify.zip

### 2. Configure Spanner CLI and create Shopify Database

In [1]:
%env CLI=/home/jupyter/kimble-munroe/spanner-cli-0.9.9/spanner-cli
%env SPAN_PROJECT=speedy-volt-324118
%env SPAN_INSTANCE=span
%env SPAN_DATABASE=span

env: CLI=/home/jupyter/kimble-munroe/spanner-cli-0.9.9/spanner-cli
env: SPAN_PROJECT=speedy-volt-324118
env: SPAN_INSTANCE=span
env: SPAN_DATABASE=span


In [2]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

In [3]:
CLI_CONNECT="$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE"

In [4]:
!{CLI_CONNECT} -e "create database shopify"

In [5]:
%env SPAN_DATABASE=shopify

env: SPAN_DATABASE=shopify


In [6]:
!$CLI -p $SPAN_PROJECT -i $SPAN_INSTANCE -d $SPAN_DATABASE -e "show tables"

In [7]:
!{CLI_CONNECT} -f /home/jupyter/kimble-munroe/spanner_shopify/create_tables.sql

In [8]:
!{CLI_CONNECT} -e "show tables"

Tables_in_shopify
apps
apps_categories
categories
key_benefits
pricing_plan_features
pricing_plans
reviews


In [9]:
!{CLI_CONNECT} -e "show create table key_benefits"

Table	Create Table
key_benefits	CREATE TABLE key_benefits (
  app_id STRING(50),
  title STRING(2000),
  description STRING(MAX),
) PRIMARY KEY(app_id, title)


In [10]:
!{CLI_CONNECT} -e "show create table apps"

Table	Create Table
apps	CREATE TABLE apps (
  id STRING(50),
  url STRING(1000),
  title STRING(2000),
  developer STRING(100),
  developer_link STRING(1000),
  icon STRING(1000),
  rating FLOAT64,
  reviews_count INT64,
) PRIMARY KEY(id)


In [11]:
!{CLI_CONNECT} -e "show create table pricing_plans"

Table	Create Table
pricing_plans	CREATE TABLE pricing_plans (
  id STRING(50),
  app_id STRING(50),
  title STRING(200),
  price FLOAT64,
) PRIMARY KEY(id)


In [12]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
) PRIMARY KEY(app_id, category_id)


In [13]:
!{CLI_CONNECT} -e "show create table pricing_plan_features"

Table	Create Table
pricing_plan_features	CREATE TABLE pricing_plan_features (
  pricing_plan_id STRING(50),
  app_id STRING(50),
  feature STRING(MAX),
) PRIMARY KEY(pricing_plan_id, app_id, feature)


In [14]:
!{CLI_CONNECT} -e "show create table categories"

Table	Create Table
categories	CREATE TABLE categories (
  id STRING(50),
  title STRING(2000),
) PRIMARY KEY(id)


In [15]:
!{CLI_CONNECT} -e "show create table reviews"

Table	Create Table
reviews	CREATE TABLE reviews (
  app_id STRING(50),
  author STRING(250),
  rating INT64,
  posted_at DATE,
) PRIMARY KEY(app_id, author)


### 3. Populate tables

In [16]:
from google.cloud import spanner
from google.cloud.spanner_v1 import param_types
import math, pandas as pd

In [17]:
span_instance='span'
span_database='shopify'
dir_path = '/home/jupyter/kimble-munroe/spanner_shopify'
spanner_client = spanner.Client()
instance = spanner_client.instance(span_instance)
database = instance.database(span_database)

In [18]:
def split_dataframe(df, split_size): 
    splits = list()
    num_splits = math.ceil(len(df) / split_size) 
    for i in range(0, len(df), split_size):
        df_subset = df.iloc[i:i + split_size]
        splits.append(df_subset)
    return splits

In [19]:
table = 'apps'
df = pd.read_csv(dir_path + '/apps.csv', sep=',', header=0, lineterminator='\n')

df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('id', 'url', 'title', 'developer', 'developer_link', 'icon', 'rating', 'reviews_count'),
            values=rows
        )

inserting 1000 rows into apps
inserting 1000 rows into apps
inserting 1000 rows into apps
inserting 547 rows into apps


In [20]:
table = 'apps_categories'
df = pd.read_csv(dir_path + '/apps_categories.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)
    
    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'category_id'),
            values=rows
        )

inserting 1000 rows into apps_categories
inserting 1000 rows into apps_categories
inserting 1000 rows into apps_categories
inserting 1000 rows into apps_categories
inserting 1000 rows into apps_categories
inserting 383 rows into apps_categories


In [21]:
table = 'categories'
df = pd.read_csv(dir_path + '/categories.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()
print('inserting ' + str(len(rows)) + ' rows into ' + table)
    
with database.batch() as batch:
    batch.insert(
        table=table,
        columns=('id', 'title'),
        values=rows
    )

inserting 12 rows into categories


In [22]:
table = 'key_benefits'
df = pd.read_csv(dir_path + '/key_benefits.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'title', 'description'),
            values=rows
        )

inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 1000 rows into key_benefits
inserting 541 rows into key_benefits


In [23]:
table = 'pricing_plans'
df = pd.read_csv(dir_path + '/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('id', 'app_id', 'title', 'price'),
            values=rows
        )

inserting 1000 rows into pricing_plans
inserting 1000 rows into pricing_plans
inserting 1000 rows into pricing_plans
inserting 1000 rows into pricing_plans
inserting 1000 rows into pricing_plans
inserting 1000 rows into pricing_plans
inserting 275 rows into pricing_plans


In [24]:
table = 'pricing_plan_features'
df = pd.read_csv(dir_path + '/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('pricing_plan_id', 'app_id', 'feature'),
            values=rows
        )

inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 1000 rows into pricing_plan_features
inserting 270 rows into pricing_plan_features


In [25]:
table = 'reviews'
df = pd.read_csv(dir_path + '/reviews.csv', sep=',', header=0, lineterminator='\n')
df_splits = split_dataframe(df, split_size=1000)
print(df[:5])

for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'author', 'rating', 'posted_at'),
            values=rows
        )

                                 app_id                  author  rating  \
0  00289a9f-9f12-45b1-963b-67e78403f7c7            Hellobeanies       5   
1  00289a9f-9f12-45b1-963b-67e78403f7c7                 Pdmovie       1   
2  00289a9f-9f12-45b1-963b-67e78403f7c7             Mommymakeup       1   
3  00289a9f-9f12-45b1-963b-67e78403f7c7  The King Herb Enhancer       5   
4  00289a9f-9f12-45b1-963b-67e78403f7c7            Training Top       4   

    posted_at  
0  2017-09-01  
1  2016-10-09  
2  2016-11-17  
3  2016-12-01  
4  2018-05-02  
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into

In [26]:
!{CLI_CONNECT} -e "select count(*) from apps"


3547


In [27]:
!{CLI_CONNECT} -e "select count(*) from apps_categories"


5383


In [28]:
!{CLI_CONNECT} -e "select count(*) from categories"


12


In [29]:
!{CLI_CONNECT} -e "select count(*) from key_benefits"


9541


In [30]:
!{CLI_CONNECT} -e "select count(*) from pricing_plans"


6275


In [31]:
!{CLI_CONNECT} -e "select count(*) from pricing_plan_features"


16270


In [32]:
def insert_app(transaction):

    row_ct = transaction.execute_update(
        "INSERT INTO apps (id, url, title, developer, developer_link, icon, reviews_count) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'https://apps.shopify.com/watchlist?surface_detail=inventory-management&surface_inter_position=1&surface_intra_position=9&surface_type=category'," 
          "'Back in Stock Product Alerts', "
          "'Swym Corporation', "
          "'https://apps.shopify.com/partners/developer-ca6a967f09890f68',"
          "'https://apps.shopifycdn.com/listing_images/9905a4c8f22cb4a3b0c32af55a58ec21/icon/e6d46a7e5e1df375d542d033aae80459.png?height=72&width=72',"
           "0)"
    )
    
    print("{} apps record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO apps_categories (app_id, category_id) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '737ad50051083aa051d127a53b3ac0da')"
    )
    
    print("{} apps_categories record(s) inserted.".format(row_ct))
    
    
    row_ct = transaction.execute_update(
        "INSERT INTO key_benefits (app_id, title, description) "
        "VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', "
          "'Drive Back In Stock Sales', 'Back in stock, out of stock, pre order & restock alerts')"
    )
    
    print("{} key_benefits record(s) inserted.".format(row_ct))
    
    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plans (id, app_id, title, price) "
        "VALUES ('961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', '683d06af-14c7-4733-9bde-ec5b699af996', "
                "'Free Trial', 14.99)"
    )
    
    print("{} pricing_plans record(s) inserted.".format(row_ct))
    

    row_ct = transaction.execute_update(
        "INSERT INTO pricing_plan_features (app_id, pricing_plan_id, feature) "
        " VALUES ('683d06af-14c7-4733-9bde-ec5b699af996', '961a2751-1dbd-4f0f-8248-9fe6a6efd4ab', "
                 "'Up to 250 alert requests/mo')"
    )
    
    print("{} pricing_plan_features record(s) inserted.".format(row_ct))
      
database.run_in_transaction(insert_app)

1 apps record(s) inserted.
1 apps_categories record(s) inserted.
1 key_benefits record(s) inserted.
1 pricing_plans record(s) inserted.
1 pricing_plan_features record(s) inserted.


In [33]:
def update_price(transaction):
    row_ct = transaction.execute_update(
            "UPDATE pricing_plans "
            "SET price = price / 2 "
            "WHERE price > 0"
        )

    print("{} pricing_plans record(s) updated.".format(row_ct))

database.run_in_transaction(update_price)

4780 pricing_plans record(s) updated.


In [34]:
def delete_app(transaction):
        row_ct = transaction.execute_update(
            "DELETE FROM apps WHERE id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )

        print("{} apps record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM apps_categories WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} apps_categories record(s) deleted.".format(row_ct))
        
        row_ct = transaction.execute_update(
            "DELETE FROM key_benefits WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} key_benefits record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plans WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} pricing_plans record(s) deleted.".format(row_ct)) 
        
        
        row_ct = transaction.execute_update(
            "DELETE FROM pricing_plan_features WHERE app_id = '683d06af-14c7-4733-9bde-ec5b699af996'"
        )
        
        print("{} pricing_plan_features record(s) deleted.".format(row_ct))

database.run_in_transaction(delete_app)

1 apps record(s) deleted.
1 apps_categories record(s) deleted.
1 key_benefits record(s) deleted.
1 pricing_plans record(s) deleted.
1 pricing_plan_features record(s) deleted.


In [35]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [36]:
!{CLI_CONNECT} -e "show create table apps_categories"

Table	Create Table
apps_categories	CREATE TABLE apps_categories (
  app_id STRING(50),
  category_id STRING(50),
  CONSTRAINT apps_cat_app_id_fk FOREIGN KEY(app_id) REFERENCES apps(id),
) PRIMARY KEY(app_id, category_id)


In [37]:
!{CLI_CONNECT} -e "CREATE INDEX apps_rating_idx ON apps(rating)"

### begin project 3 work

In [38]:
!{CLI_CONNECT} -e "select count(*) from reviews"


124601


In [39]:
# Add 100,000 fake reviews to the reviews tables
from numpy import random
fakey = [["6286d4cd-3fcb-4ee3-bb3c-de051c28b83c", "Homer Simpson " + str(i), random.randint(5), "2021-09-20"] for i in range(100000)]
df = pd.DataFrame(fakey)
print(df[:5])

table = 'reviews'

df_splits = split_dataframe(df, split_size=1000)
for split in df_splits:
    rows = split.values.tolist()
    print('inserting ' + str(len(rows)) + ' rows into ' + table)

    with database.batch() as batch:
        batch.insert(
            table=table,
            columns=('app_id', 'author', 'rating', 'posted_at'),
            values=rows
        )

                                      0                1  2           3
0  6286d4cd-3fcb-4ee3-bb3c-de051c28b83c  Homer Simpson 0  1  2021-09-20
1  6286d4cd-3fcb-4ee3-bb3c-de051c28b83c  Homer Simpson 1  2  2021-09-20
2  6286d4cd-3fcb-4ee3-bb3c-de051c28b83c  Homer Simpson 2  2  2021-09-20
3  6286d4cd-3fcb-4ee3-bb3c-de051c28b83c  Homer Simpson 3  3  2021-09-20
4  6286d4cd-3fcb-4ee3-bb3c-de051c28b83c  Homer Simpson 4  1  2021-09-20
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserting 1000 rows into reviews
inserti

In [40]:
from statistics import mean

def update_apps(transaction):
    
    avg = sum(df[:][2].to_numpy()) / 100000
    # avg = 2.00136
    
    row_ct = transaction.execute_update(
            "UPDATE apps "
            "SET rating = 2.00136 "
            "WHERE id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"
    )
    
    print("{} apps record(s) updated.".format(row_ct))
    
    row_ct = transaction.execute_update(
            "UPDATE apps "
            "SET reviews_count = 100000 "
            "WHERE id = '6286d4cd-3fcb-4ee3-bb3c-de051c28b83c'"
    )
    
    print("{} apps record(s) updated.".format(row_ct))


database.run_in_transaction(update_apps)

1 apps record(s) updated.
1 apps record(s) updated.


In [41]:
!{CLI_CONNECT} -e "CREATE INDEX reivews_author_idx ON reviews(author)"
# added index to author column on the reviews table, CPU time: 15.74 ms > 14.85 ms

In [43]:
!{CLI_CONNECT} -e "ALTER TABLE key_benefits \
  ADD CONSTRAINT apps_title_kb_title_fk FOREIGN KEY (title) REFERENCES apps (title)"

ERROR: rpc error: code = FailedPrecondition desc = Found uniqueness violation: [Advanced Cash on Delivery,d3198eb7-9077-4f82-a9bf-259925187298] [Advanced Cash on Delivery,f64300ac-c011-4eaa-be69-a6a949950c42].


In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"

In [ ]:
!{CLI_CONNECT} -e "ALTER TABLE apps_categories \
  ADD CONSTRAINT apps_cat_app_id_fk FOREIGN KEY (app_id) REFERENCES apps (id)"